In [1]:
#Importa librerias
import pandas as pd
from statsforecast import StatsForecast
import os

c:\Users\franc\miniconda3\envs\ta\lib\site-packages\statsforecast\core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#Leer los datos
df = pd.read_csv(os.path.join("refined", "top_aerolinea.csv"))

In [26]:
#Convierte el campo fecha a tipo datetime
df["Fecha"] = pd.to_datetime(df["Fecha"])
df.dtypes

In [28]:
#Ordena el dataframe por fecha
df = df.sort_values("Fecha")

In [36]:
#Se filtran las empresas que NO tengan datos hasta el 2022
ind_empresas = df.loc[df.Fecha.dt.year == 2022, "Nombre_Empresa_clean"].unique()
df = df[df.Nombre_Empresa_clean.isin(ind_empresas)].copy()

In [37]:
#Se toman las top 10 empresas con mayor trafico aereo
top_10 = list(df.groupby("Nombre_Empresa_clean")["pasajeros"].sum().sort_values(ascending=False)[:10].index)
ind = df.Nombre_Empresa_clean.isin(top_10)
df10= df[ind].copy()

In [39]:
#Se crea un nuevo dataframe para el entrenamiento
Y_df = pd.DataFrame()
Y_df["ds"] = pd.to_datetime(df10["Fecha"])
Y_df["unique_id"] = df10["Nombre_Empresa_clean"]
Y_df["y"] = df10["pasajeros"]

In [40]:
#Se visualizan las series de tiempo
StatsForecast.plot(Y_df)

In [41]:
#Se importan diferentes modelos de series de tiempo
from statsforecast.models import (
    AutoARIMA,
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive
)

models = [
    AutoARIMA(season_length=12),
    HoltWinters(),
    Croston(),
    SeasonalNaive(season_length=12),
    HistoricAverage(),
    DOT(season_length=12)
]

In [42]:
# Se instancian los modelos
sf = StatsForecast(
    df=Y_df, 
    models=models,
    freq='M', 
    n_jobs=1,
    fallback_model = SeasonalNaive(season_length=12)
)

In [43]:
#Se genera el forecast / entrenan los modelos
forecasts_df = sf.forecast(h=12, level=[95])
forecasts_df.head()

,ds,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95,HoltWinters,HoltWinters-lo-95,HoltWinters-hi-95,CrostonClassic,SeasonalNaive,SeasonalNaive-lo-95,SeasonalNaive-hi-95,HistoricAverage,HistoricAverage-lo-95,HistoricAverage-hi-95,DynamicOptimizedTheta,DynamicOptimizedTheta-lo-95,DynamicOptimizedTheta-hi-95
unique_id,,,,,,,,,,,,,,,,,
AEROGAL,2022-12-31,45232.328125,16618.015625,73846.640625,30891.0,-35855.703125,97637.703125,39905.992188,30891.0,-35855.703125,97637.703125,42367.53125,-31689.949219,116425.007812,35573.324219,11063.222656,64764.382812
AEROGAL,2023-01-31,45854.851562,6503.955078,85205.750000,29794.0,-36952.703125,96540.703125,39905.992188,29794.0,-36952.703125,96540.703125,42367.53125,-31689.949219,116425.007812,31814.804688,-9872.083008,70865.765625
AEROGAL,2023-02-28,44831.800781,-557.322876,90220.929688,37840.0,-28906.703125,104586.703125,39905.992188,37840.0,-28906.703125,104586.703125,42367.53125,-31689.949219,116425.007812,31786.410156,-22913.322266,80238.539062
AEROGAL,2023-03-31,44831.800781,-2867.209229,92530.812500,35354.0,-31392.703125,102100.703125,39905.992188,35354.0,-31392.703125,102100.703125,42367.53125,-31689.949219,116425.007812,34367.347656,-22366.675781,90737.507812
AEROGAL,2023-04-30,44831.800781,-5070.289062,94733.890625,44152.0,-22594.703125,110898.703125,39905.992188,44152.0,-22594.703125,110898.703125,42367.53125,-31689.949219,116425.007812,40513.925781,-27397.683594,104353.445312


In [44]:
#Se plotean las series de tiempo con un forecast de cada uno de los modelos propuestos
sf.plot(Y_df,forecasts_df)

In [45]:
#Se realiza crossvalidation para evaluar el desempeño de los modelos
crossvaldation_df = sf.cross_validation(
    df=Y_df,
    h=12,
    step_size=12,
    n_windows=2
  )

In [46]:
#crossvaldation_df.head()

,ds,cutoff,y,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta
unique_id,,,,,,,,,
AEROGAL,2020-12-31,2020-11-30,29004.0,45197.000000,76376.0,60350.859375,76376.0,43994.386719,25658.101562
AEROGAL,2021-01-31,2020-11-30,17470.0,45276.882812,59042.0,60350.859375,59042.0,43994.386719,22601.978516
AEROGAL,2021-02-28,2020-11-30,14340.0,48213.222656,71644.0,60350.859375,71644.0,43994.386719,22472.291016
AEROGAL,2021-03-31,2020-11-30,12768.0,54030.074219,114054.0,60350.859375,114054.0,43994.386719,25968.740234
AEROGAL,2021-04-30,2020-11-30,14926.0,60443.082031,161559.0,60350.859375,161559.0,43994.386719,32407.011719


In [47]:
#Se define una funcion para la evaluacion de los modelos
from datasetsforecast.losses import mse, mae, rmse,mape

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [48]:
#Se imprime el desempeño de los disintos modelos
evaluation_df = evaluate_cross_validation(crossvaldation_df, mape)
evaluation_df.head()

,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta,best_model
unique_id,,,,,,,
AEROGAL,90.598233,160.934871,120.647148,160.934871,74.107701,47.315378,DynamicOptimizedTheta
AEROREPUBLICA,36.601825,56.774101,13.074430,56.774101,12.960025,41.924111,HistoricAverage
AMERICAN,32.796502,44.781168,26.209954,44.781168,66.017121,28.253266,CrostonClassic
AVIANCA,16.292121,54.884807,17.724130,54.884807,39.946412,33.181839,AutoARIMA
COPA,30.010505,47.413006,46.089038,47.413006,83.734456,27.892220,DynamicOptimizedTheta


In [49]:
#Se define una variable para tomar el mejor modelo para cada serie de tiempo
def get_best_model_forecast(forecasts_df, evaluation_df):
    df = forecasts_df.set_index('ds', append=True).stack().to_frame().reset_index(level=2) # Wide to long 
    df.columns = ['model', 'best_model_forecast'] 
    df = df.join(evaluation_df[['best_model']])
    df = df.query('model.str.replace("-lo-90|-hi-90", "", regex=True) == best_model').copy()
    df.loc[:, 'model'] = [model.replace(bm, 'best_model') for model, bm in zip(df['model'], df['best_model'])]
    df = df.drop(columns='best_model').set_index('model', append=True).unstack()
    df.columns = df.columns.droplevel()
    df = df.reset_index(level=1)
    return df

In [50]:
#Se genera inferencia de cada serie de tiempo con su mejor modelo correspondiente
prod_forecasts_df = get_best_model_forecast(forecasts_df, evaluation_df)
prod_forecasts_df.head()

model,ds,best_model
unique_id,,
AEROGAL,2022-12-31,35573.324219
AEROGAL,2023-01-31,31814.804688
AEROGAL,2023-02-28,31786.410156
AEROGAL,2023-03-31,34367.347656
AEROGAL,2023-04-30,40513.925781


In [51]:
#Se plotea cada serie de tiempo con su inferencia correspondiente
sf.plot(Y_df, prod_forecasts_df, level=[95])